# Pangeo TEM Workload

This is a notebook for exploring a simplified version of the example in https://github.com/dcherian/ncar-challenge-suite/blob/main/tem.ipynb.

See https://github.com/tomwhite/cubed/issues/145

In [1]:
import cubed
import cubed.array_api as xp
import cubed.random

import pandas as pd
import xarray as xr

In [2]:
dims = ("time", "level", "lat", "lon")
# 1 is number of years, adjust to make bigger, 
# full dataset is 60-ish years.
# shape = (1 * 365 * 24, 37, 72, 1440)
shape = (1 * 31 * 24, 37, 72, 144)  # make smaller while exploring
chunks = (24, 15, -1, -1)

spec = cubed.Spec(max_mem=1_000_000_000)

In [3]:
ds = xr.Dataset(
    {
        "U": (dims, cubed.random.random(shape, chunks=chunks, spec=spec)),
        "V": (dims, cubed.random.random(shape, chunks=chunks, spec=spec)),
        "W": (dims, cubed.random.random(shape, chunks=chunks, spec=spec)),
        "T": (dims, cubed.random.random(shape, chunks=chunks, spec=spec)),
    },
    coords={"time": pd.date_range("2001-01-01", periods=shape[0], freq="H")},
)
ds

<xarray.Dataset>
Dimensions:  (time: 744, level: 37, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 ... 2001-01-31T23:00:00
Dimensions without coordinates: level, lat, lon
Data variables:
    U        (time, level, lat, lon) float64 ...
    V        (time, level, lat, lon) float64 ...
    W        (time, level, lat, lon) float64 ...
    T        (time, level, lat, lon) float64 ...

In [4]:
ds.U

<xarray.DataArray 'U' (time: 744, level: 37, lat: 72, lon: 144)>
cubed.Array<array-004, shape=(744, 37, 72, 144), dtype=float64, chunks=((24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24), (15, 15, 7), (72,), (144,))>
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 ... 2001-01-31T23:00:00
Dimensions without coordinates: level, lat, lon

In [5]:
# Note: we actually want skipna=True, but this isn't implemented in xarray yet
# see https://github.com/pydata/xarray/issues/7243
zonal_means = ds.mean("lon", skipna=False)
zonal_means

<xarray.Dataset>
Dimensions:  (time: 744, level: 37, lat: 72)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 ... 2001-01-31T23:00:00
Dimensions without coordinates: level, lat
Data variables:
    U        (time, level, lat) float64 ...
    V        (time, level, lat) float64 ...
    W        (time, level, lat) float64 ...
    T        (time, level, lat) float64 ...

In [6]:
anomaly = ds - zonal_means
anomaly['uv'] = anomaly.U*anomaly.V
anomaly['vt'] = anomaly.V*anomaly.T
anomaly['uw'] = anomaly.U*anomaly.W
anomaly

<xarray.Dataset>
Dimensions:  (time: 744, level: 37, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 ... 2001-01-31T23:00:00
Dimensions without coordinates: level, lat, lon
Data variables:
    U        (time, level, lat, lon) float64 ...
    V        (time, level, lat, lon) float64 ...
    W        (time, level, lat, lon) float64 ...
    T        (time, level, lat, lon) float64 ...
    uv       (time, level, lat, lon) float64 ...
    vt       (time, level, lat, lon) float64 ...
    uw       (time, level, lat, lon) float64 ...

In [7]:
temdiags = zonal_means.merge(anomaly[['uv','vt','uw']].mean("lon", skipna=False))  # actually want skipna=True
temdiags

<xarray.Dataset>
Dimensions:  (time: 744, level: 37, lat: 72)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 ... 2001-01-31T23:00:00
Dimensions without coordinates: level, lat
Data variables:
    U        (time, level, lat) float64 ...
    V        (time, level, lat) float64 ...
    W        (time, level, lat) float64 ...
    T        (time, level, lat) float64 ...
    uv       (time, level, lat) float64 ...
    vt       (time, level, lat) float64 ...
    uw       (time, level, lat) float64 ...

In [8]:
temdiags_mean = temdiags.groupby('time.dayofyear').mean(skipna=False)
temdiags_mean

<xarray.Dataset>
Dimensions:    (dayofyear: 31, level: 37, lat: 72)
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 9 ... 24 25 26 27 28 29 30 31
Dimensions without coordinates: level, lat
Data variables:
    U          (dayofyear, level, lat) float64 ...
    V          (dayofyear, level, lat) float64 ...
    W          (dayofyear, level, lat) float64 ...
    T          (dayofyear, level, lat) float64 ...
    uv         (dayofyear, level, lat) float64 ...
    vt         (dayofyear, level, lat) float64 ...
    uw         (dayofyear, level, lat) float64 ...

In [9]:
temdiags_mean["uv"].values

array([[[ 1.92543754e-03,  8.62170647e-04,  1.67535136e-03, ...,
         -2.50751881e-03,  2.46263741e-03, -4.70656500e-04],
        [-9.69165572e-04, -9.19808903e-04, -8.64318054e-04, ...,
          9.10300914e-04,  7.68390560e-04, -9.60456448e-04],
        [ 6.77793313e-04,  2.03099229e-03,  1.44811866e-03, ...,
          2.01528240e-03, -1.03220836e-03,  1.94811893e-03],
        ...,
        [ 2.14917057e-03,  1.47596143e-04, -7.10794437e-04, ...,
         -2.50238921e-04, -1.38891106e-03,  2.17674787e-03],
        [ 1.19509492e-03, -7.62831655e-04, -1.08836927e-03, ...,
         -1.45508192e-03,  7.55372268e-05,  2.78225531e-04],
        [-4.35496517e-03,  1.28230145e-03,  4.80595941e-04, ...,
          2.88439879e-04,  2.96332222e-03,  7.91863232e-04]],

       [[ 8.82986835e-04,  1.54489743e-03, -1.49333807e-03, ...,
          5.29418813e-04,  1.63187013e-03, -2.07224193e-04],
        [ 9.55443785e-04,  2.54465546e-03, -9.19900311e-04, ...,
         -1.39288060e-03,  2.21651069e